In [1]:
import tensorflow as tf
import numpy as np
import random
from sklearn.preprocessing import LabelEncoder

from unidecode import unidecode 

import colorama
from colorama import Fore, Style

c:\users\secol\appdata\local\programs\python\python36\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Base de conhecimento

In [2]:
base_conhecimento = {
    'base' : [
        {
            'label' : 'INIT',
            'X'     : ['Olá', 'Oi', 'bom dia', 'boa tarde', 'boa noite', 'como vai?', 'tem alguem ai?'],
            'Y'     : ['olá', 'como vai?', 'como posso ajudar?']
        },
        {
            'label' : 'END',
            'X'     : ['tchau', 'até logo', 'obrigado pela ajuda'],
            'Y'     : ['Sem problemas!', 'Meu prazer!']
        },
        {
            'label' : 'BOT_INFO',
            'X'     : ['qual e o seu nome?', 'como posso te chamar?', 'com quem falo?'],
            'Y'     : ['Você pode me chamar de rose!', 'Meu nome e rose!', 'Me chamo rose!']
        },
        {
            'label' : 'BOT_SOBRE',
            'X'     : ['quem e voce?', 'fale sobre você', 'pode se apresentar?'],
            'Y'     : ['Siri é passado! sou a Rose, seu robo assistente!', 'Meu nome é Rose, sou um chatbot!']
        },
        {
            'label' : 'BOT_AGRADECIMENTO',
            'X'     : ['obrigado', 'muito obrigado'],
            'Y'     : ['Foi um prazer ajudar!', 'Obrigado!']
        },
        {
            'label' : 'BOT_AGENDAMENTO',
            'X'     : ['quero agendar um exame', 'gostaria de saber sobre a disponibilidade de agenda para um exame', 'posso agendar um exame?', 'preciso realizar um agendamento de exame'],
            'Y'     : ['Ok, qual exame precisa agendar?', 'Certo! Preciso de mais informações. Qual exame gostaria de agendar?']
        }
    ]
}

## Trabalhando o dado

In [3]:
x_train = list()
y_train = list()
labels = list()
target = dict()

for item in base_conhecimento['base']:
    for x in item['X']:
        x_train.append(unidecode(x.lower()))
        y_train.append(item[ 'label'])
    
    if item['label'] not in labels:
        labels.append(item['label'])
        
    target[item['label']] = item['Y']
    
n_classes = len(labels)

In [4]:
x_train[:5]

['ola', 'oi', 'bom dia', 'boa tarde', 'boa noite']

In [5]:
y_train[:5]

['INIT', 'INIT', 'INIT', 'INIT', 'INIT']

In [6]:
labels[:5]

['INIT', 'END', 'BOT_INFO', 'BOT_SOBRE', 'BOT_AGRADECIMENTO']

In [7]:
target

{'INIT': ['olá', 'como vai?', 'como posso ajudar?'],
 'END': ['Sem problemas!', 'Meu prazer!'],
 'BOT_INFO': ['Você pode me chamar de rose!',
  'Meu nome e rose!',
  'Me chamo rose!'],
 'BOT_SOBRE': ['Siri é passado! sou a Rose, seu robo assistente!',
  'Meu nome é Rose, sou um chatbot!'],
 'BOT_AGRADECIMENTO': ['Foi um prazer ajudar!', 'Obrigado!'],
 'BOT_AGENDAMENTO': ['Ok, qual exame precisa agendar?',
  'Certo! Preciso de mais informações. Qual exame gostaria de agendar?']}

### Criando rótulos para cada classe

In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)

In [9]:
y_train[:5]

array([5, 5, 5, 5, 5], dtype=int64)

In [10]:
encoder.classes_

array(['BOT_AGENDAMENTO', 'BOT_AGRADECIMENTO', 'BOT_INFO', 'BOT_SOBRE',
       'END', 'INIT'], dtype='<U17')

### Criando uma nova estrutura

In [11]:
vocab_size = 25
embedding_dim = 32
max_len = 50
unk_token = '<UNK>'

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=unk_token)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, truncating='post', maxlen=max_len)

words_idx = tokenizer.word_index

In [12]:
sequences[:5]

[[14], [15], [16, 17], [6, 18], [6, 19]]

In [13]:
pad_sequences[:5]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 14],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 15],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        16, 17],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         6, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

## Modelo

In [14]:
def build_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
    model.add(tf.keras.layers.GlobalAveragePooling1D())
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [26]:
model = build_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 32)            800       
_________________________________________________________________
global_average_pooling1d_4 ( (None, 32)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_17 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_18 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_19 (Dense)             (None, 6)                 102       
Total params: 3,542
Trainable params: 3,542
Non-trainable params: 0
____________________________________________________

In [27]:
history = model.fit(pad_sequences, np.array(y_train), epochs=250, verbose=True)

Epoch 1/250
22/22 [==============================] - 0s 5ms/sample - loss: 1.7909 - acc: 0.2273
Epoch 2/250
22/22 [==============================] - 0s 45us/sample - loss: 1.7889 - acc: 0.3182
Epoch 3/250
22/22 [==============================] - 0s 91us/sample - loss: 1.7874 - acc: 0.3182
Epoch 4/250
22/22 [==============================] - 0s 45us/sample - loss: 1.7859 - acc: 0.3182
Epoch 5/250
22/22 [==============================] - 0s 91us/sample - loss: 1.7846 - acc: 0.3182
Epoch 6/250
22/22 [==============================] - 0s 91us/sample - loss: 1.7831 - acc: 0.3182
Epoch 7/250
22/22 [==============================] - 0s 91us/sample - loss: 1.7816 - acc: 0.3182
Epoch 8/250
22/22 [==============================] - 0s 91us/sample - loss: 1.7802 - acc: 0.3182
Epoch 9/250
22/22 [==============================] - 0s 46us/sample - loss: 1.7785 - acc: 0.3182
Epoch 10/250
22/22 [==============================] - 0s 91us/sample - loss: 1.7767 - acc: 0.3182
Epoch 11/250
22/22 [==========

## Testando a Rose

In [28]:
def chatbot():
    max_len = 50
    
    print(f'{Fore.RED}Chatbot ativo, envie "sair" para finalizar...{Style.RESET_ALL}')
    
    while True:
        print(f'{Fore.GREEN}Usuário: {Style.RESET_ALL}', end=' ')
        texto = unidecode(input().lower())
        
        if texto == 'sair':
            break
            
        result = model.predict(tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([texto]), truncating='post', maxlen=max_len))
        label = encoder.inverse_transform([np.argmax(result)])

        responses = target.get(label[0], None)
        
        if responses == None:
            print(f'{Fore.RED}Oooops...{Style.RESET_ALL}')
            continue
        
        response = np.random.choice(responses)
        
        print(f'{Fore.RED}Rose: {Style.RESET_ALL}{response}')
        
    print(f'{Fore.RED}Chatbot offline...{Style.RESET_ALL}')

In [29]:
chatbot()

Chatbot ativo, envie "sair" para finalizar...
Usuário:  

 ola


Rose: como vai?
Usuário:  

 tudo bem>


Rose: Sem problemas!
Usuário:  

...

Rose: Certo! Preciso de mais informações. Qual exame gostaria de agendar?
Usuário:  

 como você se chama?


Rose: Meu nome é Rose, sou um chatbot!
Usuário:  

 sair


Chatbot offline...
